In [2]:
import os
import pandas as pd
import numpy as np
import datetime
import scipy
from scipy.stats import skew, kurtosis
import tensorflow as tf
import pickle

In [8]:

def shuffle(X, Y):
  print("Shuffling data")
  indices = range(len(X))
  indices = tf.random.shuffle(indices)

  X = tf.gather(X, indices)
  Y = tf.gather(Y, indices)

  print("[Done] Shuffling data")
  return X, Y

In [5]:
with open("../data/pickles/data_slidingwindowTrue10_samplingrate20_df_BU4707.p", 'rb') as data:
    d = pickle.load(data)

In [14]:
X0 = np.array(d["X"])
Y0 = np.array(d["Y"])
X0.shape, Y0.shape

((11230, 200, 3), (11230, 1))

In [15]:
X0, Y0 = shuffle(X0, Y0)

Shuffling data
[Done] Shuffling data


In [20]:
with open("../data/pickles/data_slidingwindowTrue10_samplingrate20_df_DC6359.p", 'rb') as data_test:
    d_test = pickle.load(data_test)

X1 = np.array(d_test["X"])
Y1 = np.array(d_test["Y"])

X1, Y1 = shuffle(X1, Y1)

Shuffling data
[Done] Shuffling data


## Baseline Model 
#### 3 hidden layers (32 nodes, RELU, 32 nodes, RELU, 16 nodes, RELU)  
#### output layer uses sigmoid activation

In [36]:
batch_size = 32
epochs = 50
flatten = tf.keras.layers.Flatten()
# fc_layer1 = tf.keras.layers.Dense(units=1024, activation = 'relu')
# fc_layer2 = tf.keras.layers.Dense(units=768, activation = 'relu')
# fc_layer3 = tf.keras.layers.Dense(units=512, activation = 'relu')
# fc_layer4 = tf.keras.layers.Dense(units=256, activation = 'relu')
fc_layer5 = tf.keras.layers.Dense(units=128, activation = 'relu')
fc_layer6 = tf.keras.layers.Dense(units=64, activation = 'relu')
fc_layer7 = tf.keras.layers.Dense(units=32, activation = 'relu')
fc_layer8 = tf.keras.layers.Dense(units=1, activation = 'sigmoid')
# Cant use softmax at the end since it will normalize and give 1 
base_model = tf.keras.Sequential([
    flatten,
#  fc_layer1, fc_layer2, 
# fc_layer3, 
# fc_layer4,
fc_layer5,
fc_layer6,
fc_layer7,
fc_layer8
])

base_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits = False), \
                   optimizer=tf.keras.optimizers.Adam(0.001, beta_1=0.9, beta_2= 0.999), \
                   metrics=[tf.keras.metrics.Accuracy()]) 

base_model.build((X0.shape[0], X0.shape[1], X0.shape[2]))
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (11230, 600)              0         
                                                                 
 dense (Dense)               (11230, 128)              76928     
                                                                 
 dense_1 (Dense)             (11230, 64)               8256      
                                                                 
 dense_2 (Dense)             (11230, 32)               2080      
                                                                 
 dense_3 (Dense)             (11230, 1)                33        
                                                                 
Total params: 87,297
Trainable params: 87,297
Non-trainable params: 0
_________________________________________________________________


In [37]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100)
base_model.fit(X0, Y0, epochs = 1000,
               batch_size = 256, 
               verbose=1, 
               callbacks=[callback]) 

# Paper baseline model as-it-is
#                Epoch 500/500
# 961/961 [==============================] - 1s 579us/step - loss: 0.0989 - accuracy: 0.4819

Epoch 1/1000


2023-05-06 01:15:32.788092: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


44/44 [==============================] - 1s 2ms/step - loss: 0.6288 - accuracy: 0.0000e+00
Epoch 2/1000
44/44 [==============================] - 0s 2ms/step - loss: 0.5604 - accuracy: 0.0000e+00
Epoch 3/1000
44/44 [==============================] - 0s 2ms/step - loss: 0.5164 - accuracy: 0.0000e+00
Epoch 4/1000
44/44 [==============================] - 0s 2ms/step - loss: 0.4674 - accuracy: 0.0000e+00
Epoch 5/1000
44/44 [==============================] - 0s 2ms/step - loss: 0.4013 - accuracy: 0.0000e+00
Epoch 6/1000
44/44 [==============================] - 0s 2ms/step - loss: 0.3089 - accuracy: 0.0000e+00
Epoch 7/1000
44/44 [==============================] - 0s 2ms/step - loss: 0.2117 - accuracy: 0.0000e+00
Epoch 8/1000
44/44 [==============================] - 0s 2ms/step - loss: 0.1349 - accuracy: 2.6714e-04
Epoch 9/1000
44/44 [==============================] - 0s 5ms/step - loss: 0.0922 - accuracy: 0.0020
Epoch 10/1000
44/44 [==============================] - 0s 3ms/step - loss: 0.0800

In [38]:
loss, accuracy = base_model.evaluate(X1, Y1,
                                    #  batch_size = batch_size, 
                                     verbose=0)

loss,accuracy

(12.910189628601074, 0.016599111258983612)

## CNN Base Model

In [39]:
batch_size = 256
epochs = 1000
# prep_data = pd.read_csv('../data/good_again_bhas.csv')
flatten = tf.keras.layers.Flatten()
conv_layer1 = tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
conv_layer2 = tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='SAME')
dropout = tf.keras.layers.Dropout(0.5)
max_pooling = tf.keras.layers.MaxPool1D(pool_size=2)
# fc - fully connected layer
fc_layer = tf.keras.layers.Dense(units=128, activation = 'relu')
fc_layer2 = tf.keras.layers.Dense(units=1, activation = 'sigmoid')
base_model = tf.keras.Sequential([
                                  conv_layer1,  
                                  conv_layer2, 
                                  dropout, 
                                  max_pooling, 
                                  flatten, 
                                  fc_layer, 
                                  fc_layer2
                                ])

# base_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.0001), \
base_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), \
                   optimizer=tf.keras.optimizers.Adam(0.001), \
                   metrics=[tf.keras.metrics.Accuracy()]) 

In [54]:
# loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=False)
# optimizer = tf.keras.optimizers.Adam(0.001)

# for e in range(0, 5000):
#     with tf.GradientTape() as tape:
#         preds = base_model(final_arr_reshape)
#         print(preds)
        
#         # loss = loss_func(y_true=labels_arr_reshape, y_pred=preds)
#         # acc = np.sum(np.equal(labels, preds)) / 30735
#     #     print(acc)
#     # gradients = tape.gradient(loss, base_model.trainable_variables)
#     # optimizer.apply_gradients(zip(gradients, base_model.trainable_variables))
#     # print(f"Epoch: {e} | LOSS : {loss} | acc {acc}")

# # loss
            
# # total_loss += loss
# # gradients = tape.gradient(loss, model.trainable_variables)
# #     optimizer.apply_gradients(zip(gradients, model.trainable_variables))
# #     return total_loss


In [40]:
base_model.build(X0.shape)
base_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (11230, 200, 64)          640       
                                                                 
 conv1d_1 (Conv1D)           (11230, 200, 64)          12352     
                                                                 
 dropout (Dropout)           (11230, 200, 64)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (11230, 100, 64)         0         
 )                                                               
                                                                 
 flatten_1 (Flatten)         (11230, 6400)             0         
                                                                 
 dense_4 (Dense)             (11230, 128)              819328    
                                                      

In [41]:
base_model.fit(X0, Y0, epochs = epochs,
               batch_size = batch_size, 
               verbose=1) 

Epoch 1/1000
44/44 [==============================] - 2s 42ms/step - loss: 0.5924 - accuracy: 0.0000e+00
Epoch 2/1000
44/44 [==============================] - 2s 34ms/step - loss: 0.5485 - accuracy: 0.0000e+00
Epoch 3/1000
44/44 [==============================] - 2s 34ms/step - loss: 0.5297 - accuracy: 0.0000e+00
Epoch 4/1000
44/44 [==============================] - 2s 36ms/step - loss: 0.5172 - accuracy: 0.0000e+00
Epoch 5/1000
44/44 [==============================] - 2s 47ms/step - loss: 0.5116 - accuracy: 0.0000e+00
Epoch 6/1000
44/44 [==============================] - 2s 41ms/step - loss: 0.5004 - accuracy: 0.0000e+00
Epoch 7/1000
44/44 [==============================] - 1s 34ms/step - loss: 0.4895 - accuracy: 0.0000e+00
Epoch 8/1000
44/44 [==============================] - 1s 31ms/step - loss: 0.4775 - accuracy: 8.9047e-05
Epoch 9/1000
44/44 [==============================] - 2s 35ms/step - loss: 0.4639 - accuracy: 5.3428e-04
Epoch 10/1000
44/44 [==============================] - 

KeyboardInterrupt: 

In [150]:
loss, accuracy = base_model.evaluate(X1, Y1,
                                    #  batch_size = batch_size, 
                                     verbose=0)

loss, accuracy

(0.5554, 0.6346510648727417)

In [ ]:
base_model.save('../code/CNN_base_model.h5')
base_model.save('../code/CNN_base_model.pkl')

## MLP Classifier

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
clf = MLPClassifier(solver='adam', shuffle=True, random_state=1)

In [18]:
X0_dash = np.reshape(X0, (X0.shape[0], X0.shape[1] * X0.shape[2]))

clf.fit(X0_dash, Y0)
clf.get_params()

# print('Accuracy ', accuracy_score(y_test, clf.predict(x_test)))

/Users/sarah_prakriti_peters/miniconda3/envs/DL/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': 1,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [21]:
X1_dash = np.reshape(X1, (X1.shape[0], X1.shape[1] * X1.shape[2]))

In [22]:
print('Accuracy ', accuracy_score(Y1, clf.predict(X1_dash)))

Accuracy  0.6874477256642861


In [35]:
np.reshape(Y1, Y1.shape[0]).mean()

0.1478479